### How this works - Step by step explanation
1. Diagrams obtained using simulated annealing, coefficients missing (especially also after changing the diagrams to nicer forms, e.g. Hadamard edges)
2. Print matrices of those diagrams
3. Use WolframAlpha to figure out what exact number the obtained numerical values are (usually multiples of sqrt(2))
4. Multiply by the inverse of those numbers exactly (e.g. using g.scalar.add_float(0.125))
5. This gives arrays usually only consisting of ones
6. Now evaluate generated code in Mathematica -> Usually yields integer coefficient solutions
7. Multiply diagrams with those new values -> Finaly coefficients/decomposition
(8. Check decomposition)

In [1]:
import pyzx as zx
from pyzx.basicrules import fuse
from pyzx.tikz import tikz_to_graph
from fractions import Fraction
import numpy as np
import sympy as sp
from collections import defaultdict
import matplotlib.pyplot as plt
import sys

np.set_printoptions(threshold=np.inf) # print the whole matrix

In [2]:
Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
W_INP = zx.VertexType.W_INPUT
W_OUT = zx.VertexType.W_OUTPUT
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [3]:
draw_scale_default = 30

def nice_print(str, conditional=True):
    if conditional:
        print(str)

# like zx.draw but with my favorite settings already set
def nice_draw(g, conditional=True, labels=True, scale=draw_scale_default):
    if not conditional:
        return
    
    exact_string = str(g.scalar).split(" = ")[1]
    numeric_string = str(g.scalar.to_number())
    print("\t" + exact_string + " ≈ " + numeric_string)

    zx.draw(g, labels=labels, scale=scale)

def to_pure_matrix(matrix):
    # get rid of floating point error
    # change -0.0 to 0.0
    return np.around(matrix, 10) + 0.

In [4]:
def tikz_file_to_graph(path):
    with open(path, 'r') as file:
        content = file.read()
    return tikz_to_graph(content)

In [5]:
def vec_to_mathematica_str(name, vec):
    data = repr(vec).strip(u'\u200b')[6:][:-1]

    return "{}=ImportString[\"{}\",\"PythonExpression\"];".format(name, data)

In [6]:
def create_n_states(n, phase):
    goal_graph = zx.Graph()

    for i in range(n):
        cur = zx.Graph()
        
        z = cur.add_vertex(Z, qubit=1, row=0, phase=phase)
        x = cur.add_vertex(X, qubit=1, row=1, phase=Fraction(1,1))
        out = cur.add_vertex(B, qubit=1, row=2)
        cur.add_edge((z,x))
        cur.add_edge((x,out))
        cur.auto_detect_io()

        triangle = zx.generate.spider("W",1,2) + (zx.generate.spider("Z", 1, 0) @ zx.generate.identity(1))
        triangle.auto_detect_io()

        cur = cur + triangle

        goal_graph = goal_graph @ cur
    
    return goal_graph

In [7]:
orig_g = create_n_states(n=5, phase=0)

nice_draw(orig_g, labels=False)

orig_matrix = to_pure_matrix(orig_g.to_matrix())

print(orig_matrix)

	sqrt(2)^0 ≈ (1+0j)


[[32.+0.j]
 [16.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [ 2.+0.j]
 [ 1.+0.j]]


In [8]:
g_matrices = []

In [9]:
print(0)
g0 = tikz_file_to_graph("0.tikz")
g0.scalar.add_float(32)
nice_draw(g0, labels=False)
g0.auto_detect_io()
g_matrices.append(to_pure_matrix(g0.to_matrix()))
print(g0.to_matrix())

print(1)
g1 = tikz_file_to_graph("1.tikz")
g1.scalar.add_float(0.125)
g1.scalar.add_power(1)
nice_draw(g1, labels=False)
g1.auto_detect_io()
g_matrices.append(to_pure_matrix(g1.to_matrix()))
print(g1.to_matrix())

print(2)
g2 = tikz_file_to_graph("2.tikz")
g2.scalar.add_float(2)
g2.scalar.add_power(1)
nice_draw(g2, labels=False)
g2.auto_detect_io()
g_matrices.append(to_pure_matrix(g2.to_matrix()))
print(g2.to_matrix())

print(3)
g3 = tikz_file_to_graph("3.tikz")
g3.scalar.add_float(2)
g3.scalar.add_power(1)
nice_draw(g3, labels=False)
g3.auto_detect_io()
g_matrices.append(to_pure_matrix(g3.to_matrix()))
print(g3.to_matrix())

print(4)
g4 = tikz_file_to_graph("4.tikz")
g4.scalar.add_float(16)
g4.scalar.add_power(1)
nice_draw(g4, labels=False)
g4.auto_detect_io()
g_matrices.append(to_pure_matrix(g4.to_matrix()))
print(g4.to_matrix())

print(5)
g5 = tikz_file_to_graph("5.tikz")
nice_draw(g5, labels=False)
g5.auto_detect_io()
g_matrices.append(to_pure_matrix(g5.to_matrix()))
print(g5.to_matrix())

0


FileNotFoundError: [Errno 2] No such file or directory: '0.tikz'

In [24]:
mats = g_matrices

goal_str = vec_to_mathematica_str("Goal", orig_matrix.flatten())
mat_strs = []
solve_str = "Solve[m0*M0"

for i, mat in enumerate(mats):
    mat_strs.append(vec_to_mathematica_str("M" + str(i), mat))

    if i > 0:
        solve_str += " + m{}*M{}".format(i, i)

solve_str += " == Goal, {m0"

for i in range(1, len(mats)):
    solve_str += ", m{}".format(i)

solve_str += "}]"

print(goal_str)
for mat_str in mat_strs:
    print(mat_str)
print(solve_str)

Goal=ImportString["[32.+0.j, 16.+0.j, 16.+0.j,  8.+0.j, 16.+0.j,  8.+0.j,  8.+0.j,
        4.+0.j, 16.+0.j,  8.+0.j,  8.+0.j,  4.+0.j,  8.+0.j,  4.+0.j,
        4.+0.j,  2.+0.j, 16.+0.j,  8.+0.j,  8.+0.j,  4.+0.j,  8.+0.j,
        4.+0.j,  4.+0.j,  2.+0.j,  8.+0.j,  4.+0.j,  4.+0.j,  2.+0.j,
        4.+0.j,  2.+0.j,  2.+0.j,  1.+0.j]","PythonExpression"];
M0=ImportString["[[ 1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [-1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [ 1.+0.j],
       [-1.+0.j]]","PythonExpression"];
M1=ImportString["[[1.+0.j],
     

In [25]:
g_matrices = []

print(0)
g0 = tikz_file_to_graph("0.tikz")
g0.scalar.add_float(32)
g0.scalar.add_float(-6)
nice_draw(g0, labels=False)
g0.auto_detect_io()
g_matrices.append(to_pure_matrix(g0.to_matrix()))
print(g0.to_matrix())

print(1)
g1 = tikz_file_to_graph("1.tikz")
g1.scalar.add_float(0.125)
g1.scalar.add_power(1)
g1.scalar.add_float(15)
nice_draw(g1, labels=False)
g1.auto_detect_io()
g_matrices.append(to_pure_matrix(g1.to_matrix()))
print(g1.to_matrix())

print(2)
g2 = tikz_file_to_graph("2.tikz")
g2.scalar.add_float(2)
g2.scalar.add_power(1)
g2.scalar.add_float(5)
nice_draw(g2, labels=False)
g2.auto_detect_io()
g_matrices.append(to_pure_matrix(g2.to_matrix()))
print(g2.to_matrix())

print(3)
g3 = tikz_file_to_graph("3.tikz")
g3.scalar.add_float(2)
g3.scalar.add_power(1)
g3.scalar.add_float(10)
nice_draw(g3, labels=False)
g3.auto_detect_io()
g_matrices.append(to_pure_matrix(g3.to_matrix()))
print(g3.to_matrix())

print(4)
g4 = tikz_file_to_graph("4.tikz")
g4.scalar.add_float(16)
g4.scalar.add_power(1)
g4.scalar.add_float(3)
nice_draw(g4, labels=False)
g4.auto_detect_io()
g_matrices.append(to_pure_matrix(g4.to_matrix()))
print(g4.to_matrix())

print(5)
g5 = tikz_file_to_graph("5.tikz")
g5.scalar.add_float(15)
nice_draw(g5, labels=False)
g5.auto_detect_io()
g_matrices.append(to_pure_matrix(g5.to_matrix()))
print(g5.to_matrix())

0
	-192.00+0.00isqrt(2)^0 ≈ (-192+0j)


[[-6.+0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [-6.+0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [-6.+0.j]
 [ 6.-0.j]
 [-6.+0.j]
 [-6.+0.j]
 [-6.+0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [ 6.-0.j]
 [-6.+0.j]
 [ 6.-0.j]
 [-6.+0.j]
 [-6.+0.j]
 [-6.+0.j]
 [ 6.-0.j]
 [-6.+0.j]
 [-6.+0.j]
 [-6.+0.j]
 [-6.+0.j]
 [-6.+0.j]
 [-6.+0.j]
 [ 6.-0.j]]
1
	1.88+0.00isqrt(2)^1 ≈ (2.6516504294495533+0j)


[[15.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [-0.+0.j]
 [ 0.+0.j]
 [-0.+0.j]
 [-0.+0.j]
 [ 0.+0.j]
 [-0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [ 0.+0.j]]
2
	10.00+0.00isqrt(2)^1 ≈ (14.142135623730951+0j)


[[5.+0.j]
 [0.+0.j]
 [0.+0.j]
 [5.+0.j]
 [0.+0.j]
 [5.+0.j]
 [5.+0.j]
 [0.+0.j]
 [0.+0.j]
 [5.+0.j]
 [5.+0.j]
 [0.+0.j]
 [5.+0.j]
 [0.+0.j]
 [0.+0.j]
 [5.+0.j]
 [0.+0.j]
 [5.+0.j]
 [5.+0.j]
 [0.+0.j]
 [5.+0.j]
 [0.+0.j]
 [0.+0.j]
 [5.+0.j]
 [5.+0.j]
 [0.+0.j]
 [0.+0.j]
 [5.+0.j]
 [0.+0.j]
 [5.+0.j]
 [5.+0.j]
 [0.+0.j]]
3
	20.00+0.00isqrt(2)^1 ≈ (28.284271247461902+0j)


[[ 0.+0.j]
 [10.-0.j]
 [10.-0.j]
 [ 0.+0.j]
 [10.-0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [10.-0.j]
 [10.-0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [10.-0.j]
 [ 0.+0.j]
 [10.-0.j]
 [10.-0.j]
 [ 0.+0.j]
 [10.-0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [10.-0.j]
 [ 0.+0.j]
 [10.-0.j]
 [10.-0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [10.-0.j]
 [10.-0.j]
 [ 0.+0.j]
 [10.-0.j]
 [ 0.+0.j]
 [ 0.+0.j]
 [10.-0.j]]
4
	48.00+0.00isqrt(2)^1 ≈ (67.88225099390857+0j)


[[ 3.+0.j]
 [-0.+0.j]
 [ 0.-0.j]
 [-3.+0.j]
 [ 0.+0.j]
 [-3.+0.j]
 [-3.+0.j]
 [ 0.-0.j]
 [ 0.-0.j]
 [-3.+0.j]
 [-3.+0.j]
 [ 0.-0.j]
 [-3.+0.j]
 [ 0.-0.j]
 [-0.-0.j]
 [ 3.-0.j]
 [ 0.-0.j]
 [-3.+0.j]
 [-3.+0.j]
 [ 0.-0.j]
 [-3.+0.j]
 [ 0.-0.j]
 [-0.-0.j]
 [ 3.-0.j]
 [-3.+0.j]
 [ 0.-0.j]
 [-0.-0.j]
 [ 3.-0.j]
 [-0.-0.j]
 [ 3.-0.j]
 [ 3.-0.j]
 [-0.+0.j]]
5
	15.00+0.00isqrt(2)^0 ≈ (15+0j)


[[ 15.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [  0.+0.j]
 [-15.+0.j]]


In [26]:
final_matrix = np.zeros((2**5, 1), dtype=complex)

for m in g_matrices:
    final_matrix += m

print(final_matrix)

[[32.+0.j]
 [16.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [16.+0.j]
 [ 8.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [ 8.+0.j]
 [ 4.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [ 4.+0.j]
 [ 2.+0.j]
 [ 2.+0.j]
 [ 1.+0.j]]


# Phase